In [ ]:
# Install required libraries
!pip install ultralytics
!pip install pydicom
!pip install scikit-image
!pip install matplotlib pandas seaborn opencv-python --upgrade

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from tqdm.notebook import tqdm
import pydicom
from skimage import exposure
import glob
import shutil
import random
import yaml
from IPython.display import display, Image as DisplayImage
from google.colab import drive
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from PIL import Image
import time

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch
from tqdm.notebook import tqdm
import pydicom
from skimage import exposure
import glob
import shutil
import random
import yaml
from IPython.display import display, Image as DisplayImage
from google.colab import drive
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from PIL import Image
import time

# CELL 3 - MOUNT GOOGLE DRIVE & SET UP DIRECTORIES
def setup_environment():
    """
    Mount Google Drive and set up project directories

    Returns:
        dict: Dictionary containing paths to main directories
    """
    # Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Set paths
    base_dir = '/content/drive/MyDrive/LungCancerProject'
    data_dir = f'{base_dir}/data'
    models_dir = f'{base_dir}/models'
    output_dir = f'{base_dir}/output'

    # Create directories if they don't exist
    for directory in [base_dir, data_dir, models_dir, output_dir]:
        os.makedirs(directory, exist_ok=True)

    print("Directory structure created in Google Drive")

    return {
        'base_dir': base_dir,
        'data_dir': data_dir,
        'models_dir': models_dir,
        'output_dir': output_dir
    }

# Call the function to set up directories
paths = setup_environment()

In [ ]:
def preprocess_dicom(dicom_path, output_path, window_center=-600, window_width=1500):
    """
    Read a DICOM file, apply windowing, and save as JPG

    Args:
        dicom_path: Path to the DICOM file
        output_path: Path to save the processed image
        window_center: Window center for visualization (default: -600 for lungs)
        window_width: Window width for visualization (default: 1500 for lungs)

    Returns:
        str: Path to the saved image
    """
    # Read DICOM file
    dicom = pydicom.dcmread(dicom_path)

    # Get pixel array
    img = dicom.pixel_array

    # Apply windowing
    img = exposure.rescale_intensity(
        img,
        in_range=(window_center - window_width/2, window_center + window_width/2),
        out_range=(0, 255)
    )

    # Convert to uint8
    img = img.astype(np.uint8)

    # Save as JPG
    cv2.imwrite(output_path, img)
    return output_path

In [ ]:
def process_dicom_folder(input_folder, output_folder, label=None):
    """
    Process all DICOM files in a folder and convert them to images

    Args:
        input_folder: Folder containing DICOM files
        output_folder: Folder to save processed images
        label: Class label for the images (optional)

    Returns:
        list: List of paths to processed images
    """
    os.makedirs(output_folder, exist_ok=True)
    processed_images = []

    for file in tqdm(glob.glob(os.path.join(input_folder, "*.dcm")), desc=f"Processing {input_folder}"):
        filename = os.path.basename(file).replace('.dcm', '.jpg')
        output_path = os.path.join(output_folder, filename)

        preprocess_dicom(file, output_path)

        if label:
            processed_images.append((output_path, label))
        else:
            processed_images.append(output_path)

    return processed_images


In [ ]:
def create_yolo_annotations(image_path, label_id, boxes):
    """
    Create YOLO format annotations for an image

    Args:
        image_path: Path to the image
        label_id: Class ID (0-based)
        boxes: List of bounding boxes in format [x, y, width, height] (normalized)

    Returns:
        str: Path to the annotation file
    """
    img = cv2.imread(image_path)
    height, width = img.shape[:2]

    # Create annotation file path
    txt_path = image_path.replace('.jpg', '.txt').replace('/images/', '/labels/')
    os.makedirs(os.path.dirname(txt_path), exist_ok=True)

    with open(txt_path, 'w') as f:
        for box in boxes:
            # Normalize box coordinates to [0-1]
            x_center = (box[0] + box[2]/2) / width
            y_center = (box[1] + box[3]/2) / height
            w = box[2] / width
            h = box[3] / height

            # Write in YOLO format: class x_center y_center width height
            f.write(f"{label_id} {x_center} {y_center} {w} {h}\n")

    return txt_path


In [ ]:
def prepare_dataset(data_root, output_folder, labels, annotations=None, split_ratio=0.8):
    """
    Prepare dataset in YOLO format

    Args:
        data_root: Root folder containing the dataset
        output_folder: Folder to save the prepared dataset
        labels: List of class labels
        annotations: Dictionary mapping image paths to lists of bounding boxes (optional)
        split_ratio: Train/test split ratio

    Returns:
        str: Path to the dataset configuration YAML file
    """
    # Create train and val directories
    train_images_dir = os.path.join(output_folder, 'images', 'train')
    val_images_dir = os.path.join(output_folder, 'images', 'val')
    train_labels_dir = os.path.join(output_folder, 'labels', 'train')
    val_labels_dir = os.path.join(output_folder, 'labels', 'val')

    for directory in [train_images_dir, val_images_dir, train_labels_dir, val_labels_dir]:
        os.makedirs(directory, exist_ok=True)

    all_images = []

    # Process each class folder
    for idx, label in enumerate(tqdm(labels, desc="Processing class folders")):
        if os.path.isdir(os.path.join(data_root, label)):
            # Get all jpg images in the class folder
            class_images = glob.glob(os.path.join(data_root, label, "*.jpg"))
            for img_path in class_images:
                all_images.append((img_path, idx))

    # Split into train and val sets
    random.shuffle(all_images)
    split_idx = int(len(all_images) * split_ratio)
    train_images = all_images[:split_idx]
    val_images = all_images[split_idx:]

    # Process train images
    for img_path, label_id in tqdm(train_images, desc="Processing train images"):
        filename = os.path.basename(img_path)
        dest_path = os.path.join(train_images_dir, filename)
        shutil.copy(img_path, dest_path)

        # Create annotation if available
        if annotations and img_path in annotations:
            create_yolo_annotations(dest_path, label_id, annotations[img_path])
        else:
            # No annotations - use whole image as bounding box
            img = cv2.imread(img_path)
            height, width = img.shape[:2]
            create_yolo_annotations(dest_path, label_id, [[0, 0, width, height]])

    # Process val images
    for img_path, label_id in tqdm(val_images, desc="Processing val images"):
        filename = os.path.basename(img_path)
        dest_path = os.path.join(val_images_dir, filename)
        shutil.copy(img_path, dest_path)

        # Create annotation if available
        if annotations and img_path in annotations:
            create_yolo_annotations(dest_path, label_id, annotations[img_path])
        else:
            # No annotations - use whole image as bounding box
            img = cv2.imread(img_path)
            height, width = img.shape[:2]
            create_yolo_annotations(dest_path, label_id, [[0, 0, width, height]])

    # Create dataset.yaml for YOLO
    dataset_config = {
        'path': output_folder,
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(labels),
        'names': labels
    }

    yaml_path = os.path.join(output_folder, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_config, f)

    print(f"Dataset prepared in YOLO format at {output_folder}")
    print(f"Train images: {len(train_images)}")
    print(f"Val images: {len(val_images)}")
    return yaml_path

In [ ]:
def prepare_chest_ct_dataset(data_root, output_folder):
    """
    Prepare chest CT dataset specifically for the lung cancer project

    Args:
        data_root: Path to the chest CT dataset
        output_folder: Folder to save the prepared dataset

    Returns:
        str: Path to the dataset configuration YAML file
    """
    labels = ['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']

    # For this dataset, we'll assume the structure from the original notebook
    train_folder_names = [
        'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib',
        'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa',
        'normal',
        'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa'
    ]

    # Create temporary folder to store processed images by class
    temp_dir = os.path.join(output_folder, 'temp')
    os.makedirs(temp_dir, exist_ok=True)

    for folder_name, label in zip(train_folder_names, labels):
        source_folder = os.path.join(data_root, 'train', folder_name)
        target_folder = os.path.join(temp_dir, label)
        os.makedirs(target_folder, exist_ok=True)

        if os.path.isdir(source_folder):
            for img_file in tqdm(os.listdir(source_folder), desc=f"Processing {folder_name}"):
                source_path = os.path.join(source_folder, img_file)
                target_path = os.path.join(target_folder, img_file)

                # Read, resize and save the image
                img = cv2.imread(source_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (640, 640))  # YOLO default size
                    cv2.imwrite(target_path, img)

    # Also process the test data in the same way
    for label in labels:
        source_folder = os.path.join(data_root, 'test', label)
        target_folder = os.path.join(temp_dir, label)

        if os.path.isdir(source_folder):
            for img_file in tqdm(os.listdir(source_folder), desc=f"Processing test/{label}"):
                source_path = os.path.join(source_folder, img_file)
                target_path = os.path.join(target_folder, f"test_{img_file}")

                # Read, resize and save the image
                img = cv2.imread(source_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (640, 640))  # YOLO default size
                    cv2.imwrite(target_path, img)

    # Prepare the dataset in YOLO format
    yaml_path = prepare_dataset(temp_dir, output_folder, labels)

    # Cleanup temporary directory
    # shutil.rmtree(temp_dir)

    return yaml_path


In [ ]:
def download_yolo12n():
    """
    Download YOLO12n model weights if they don't exist

    Returns:
        str: Path to the downloaded model
    """
    model_path = "yolo12n.pt"
    if not os.path.exists(model_path):
        print("Downloading YOLO12n model...")
        # This is a placeholder - you'll need to replace with actual download link
        # !wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12n.pt

        # For now, we'll use YOLOv8n as a substitute
        !wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12n.pt

    return model_path

In [ ]:
def train_yolo_model(data_yaml, pretrained_weights='yolo12n.pt', epochs=100, patience=15, img_size=640, batch_size=16):
    """
    Train YOLO model on the prepared dataset

    Args:
        data_yaml: Path to the dataset YAML file
        pretrained_weights: Path to pretrained weights
        epochs: Number of training epochs
        patience: Early stopping patience
        img_size: Image size for training
        batch_size: Batch size

    Returns:
        YOLO: Trained YOLO model
    """
    # Ensure the model weights are downloaded
    model_weights = download_yolo12n()

    print(f"Loading {model_weights} model...")
    model = YOLO(model_weights)

    print(f"Training model for {epochs} epochs on {data_yaml}...")
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        patience=patience,
        device=0 if torch.cuda.is_available() else 'cpu',
        verbose=True,
        plots=True,
        save=True
    )

    print("Training complete!")
    return model

In [ ]:
def validate_yolo_model(model, data_yaml):
    """
    Validate trained YOLO model

    Args:
        model: Trained YOLO model
        data_yaml: Path to the dataset YAML file

    Returns:
        dict: Validation metrics
    """
    print("Validating model...")
    results = model.val(data=data_yaml)

    print("Validation results:")
    metrics = {
        'precision': results.box.maps[0],  # mAP@0.5
        'recall': results.box.mar[0],      # mAR@0.5
        'mAP50': results.box.map50,        # mAP@0.5
        'mAP50-95': results.box.map        # mAP@0.5:0.95
    }

    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"mAP@0.5: {metrics['mAP50']:.4f}")
    print(f"mAP@0.5:0.95: {metrics['mAP50-95']:.4f}")

    return metrics


In [ ]:
def visualize_predictions(model, test_folder, num_samples=6, conf_threshold=0.25):
    """
    Visualize model predictions on test images

    Args:
        model: Trained YOLO model
        test_folder: Folder containing test images
        num_samples: Number of images to visualize
        conf_threshold: Confidence threshold for predictions
    """
    # Get test images
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))[:num_samples]

    # Create figure
    n_cols = min(3, num_samples)
    n_rows = (num_samples + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if num_samples > 1 else [axes]

    for i, img_path in enumerate(test_images):
        if i >= num_samples:
            break

        # Run prediction
        results = model.predict(img_path, conf=conf_threshold)[0]

        # Get the original image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Plot results
        axes[i].imshow(img)

        # Draw bounding boxes and labels
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            cls_name = results.names[cls_id]

            # Draw rectangle
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False,
                                edgecolor='red', linewidth=2)
            axes[i].add_patch(rect)

            # Add label
            axes[i].text(x1, y1-5, f"{cls_name} {conf:.2f}",
                        color='white', fontsize=10,
                        bbox=dict(facecolor='red', alpha=0.7))

        axes[i].set_title(f"Predictions for {os.path.basename(img_path)}")
        axes[i].axis('off')

    # Hide empty subplots
    for i in range(len(test_images), len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def predict_single_image(model, image_path, conf_threshold=0.25):
    """
    Make prediction on a single image and visualize the results

    Args:
        model: Trained YOLO model
        image_path: Path to the image
        conf_threshold: Confidence threshold for predictions

    Returns:
        results: YOLO prediction results
    """
    # Run prediction
    results = model.predict(image_path, conf=conf_threshold)[0]

    # Get the original image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Visualize results
    plt.figure(figsize=(10, 8))
    plt.imshow(img)

    # Draw bounding boxes and labels
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        cls_name = results.names[cls_id]

        # Draw rectangle
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False,
                            edgecolor='red', linewidth=2)
        plt.gca().add_patch(rect)

        # Add label
        plt.text(x1, y1-5, f"{cls_name} {conf:.2f}",
                color='white', fontsize=10,
                bbox=dict(facecolor='red', alpha=0.7))

    plt.title(f"Predictions for {os.path.basename(image_path)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return results

In [ ]:
def create_confusion_matrix(model, data_yaml):
    """
    Create confusion matrix from model predictions

    Args:
        model: Trained YOLO model
        data_yaml: Path to the dataset YAML file
    """
    # Get validation results with confusion matrix
    results = model.val(data=data_yaml, verbose=False)

    # Extract confusion matrix
    confusion_matrix = results.confusion_matrix.matrix

    # Get class names
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    class_names = data_config['names']

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()

In [ ]:
def test_model_on_samples(model, data_yaml, num_samples=10):
    """
    Test the model on sample images from each class (cancer and non-cancer)

    Args:
        model: Trained YOLO model
        data_yaml: Path to the dataset YAML file
        num_samples: Total number of samples to test (approximately equal from each class)
    """
    print(f"Testing model on {num_samples} sample images...")

    # Load dataset configuration
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)

    # Get class names
    class_names = data_config['names']

    # Path to validation images
    val_images_dir = os.path.join(data_config['path'], data_config['val'])

    # Get labels directory
    val_labels_dir = os.path.join(data_config['path'], 'labels', 'val')

    # Dict to store images by class
    images_by_class = {cls: [] for cls in class_names}

    # Find images with their ground truth classes
    for label_file in glob.glob(os.path.join(val_labels_dir, '*.txt')):
        # Get corresponding image path
        image_name = os.path.basename(label_file).replace('.txt', '.jpg')
        image_path = os.path.join(val_images_dir, image_name)

        if not os.path.exists(image_path):
            continue

        # Read the first class from the label file (assuming one object per image)
        with open(label_file, 'r') as f:
            first_line = f.readline().strip()
            if first_line:
                try:
                    class_id = int(first_line.split()[0])
                    class_name = class_names[class_id]
                    images_by_class[class_name].append(image_path)
                except (IndexError, ValueError):
                    continue

    # If no proper labels found, just get random images
    if all(len(imgs) == 0 for imgs in images_by_class.values()):
        print("No labeled images found. Getting random samples...")
        all_images = glob.glob(os.path.join(val_images_dir, '*.jpg'))

        # For demo, we'll classify 'normal' as non-cancer and others as cancer
        cancer_classes = [cls for cls in class_names if cls != 'normal']

        # Create two groups - estimate class from filename
        for img_path in all_images:
            filename = os.path.basename(img_path).lower()

            if any(cls.lower() in filename for cls in cancer_classes):
                # Randomly assign to a cancer class
                random_cancer_class = random.choice([cls for cls in cancer_classes
                                                    if cls.lower() in filename])
                images_by_class[random_cancer_class].append(img_path)
            elif 'normal' in filename:
                images_by_class['normal'].append(img_path)

    # Count total available images
    total_available = sum(len(imgs) for imgs in images_by_class.values())
    if total_available == 0:
        print("No test images found. Please check your dataset.")
        return

    # Select samples from each class (try to balance)
    samples_per_class = max(1, num_samples // len(class_names))
    selected_images = []

    for cls, imgs in images_by_class.items():
        # Select up to samples_per_class images from this class
        cls_samples = random.sample(imgs, min(samples_per_class, len(imgs)))
        selected_images.extend([(img, cls) for img in cls_samples])

    # Shuffle the selected images
    random.shuffle(selected_images)

    # Limit to requested number of samples
    selected_images = selected_images[:num_samples]

    # Group into cancer and non-cancer for display
    cancer_samples = []
    non_cancer_samples = []

    for img_path, cls in selected_images:
        if cls == 'normal':
            non_cancer_samples.append((img_path, cls))
        else:
            cancer_samples.append((img_path, cls))

    # Testing on cancer samples
    if cancer_samples:
        print(f"\n---------- TESTING ON {len(cancer_samples)} CANCER SAMPLES ----------")
        for img_path, cls in cancer_samples:
            print(f"\nGround truth: {cls} (CANCER)")
            results = predict_single_image(model, img_path)
            # Print detection results
            boxes = results.boxes
            if len(boxes) == 0:
                print("No detections found.")
            else:
                for i, box in enumerate(boxes):
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    cls_name = results.names[cls_id]
                    cancer_status = "CANCER" if cls_name != "normal" else "NON-CANCER"
                    print(f"Detection {i+1}: {cls_name} ({cancer_status}) with confidence {conf:.2f}")

            # Small pause to let user see the results
            time.sleep(1)

    # Testing on non-cancer samples
    if non_cancer_samples:
        print(f"\n---------- TESTING ON {len(non_cancer_samples)} NON-CANCER SAMPLES ----------")
        for img_path, cls in non_cancer_samples:
            print(f"\nGround truth: {cls} (NON-CANCER)")
            results = predict_single_image(model, img_path)
            # Print detection results
            boxes = results.boxes
            if len(boxes) == 0:
                print("No detections found.")
            else:
                for i, box in enumerate(boxes):
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    cls_name = results.names[cls_id]
                    cancer_status = "CANCER" if cls_name != "normal" else "NON-CANCER"
                    print(f"Detection {i+1}: {cls_name} ({cancer_status}) with confidence {conf:.2f}")

            # Small pause to let user see the results
            time.sleep(1)

    # Summary
    print("\n---------- TEST SUMMARY ----------")
    print(f"Total samples tested: {len(selected_images)}")
    print(f"Cancer samples: {len(cancer_samples)}")
    print(f"Non-cancer samples: {len(non_cancer_samples)}")
    print("Test complete!")

In [ ]:
def create_demo_interface(model):
    """
    Create a simple interface for making predictions on new images

    Args:
        model: Trained YOLO model
    """
    from google.colab import files
    from IPython.display import clear_output

    # Function to handle file upload
    def upload_and_predict():
        clear_output()
        print("Upload an image for prediction...")
        uploaded = files.upload()

        for filename in uploaded.keys():
            print(f"Predicting on {filename}...")
            file_path = filename
            predict_single_image(model, file_path)

    # Call the function
    upload_and_predict()

In [ ]:
def process_dataset():
    """
    Process the chest CT dataset and prepare it for YOLO

    Returns:
        str: Path to the dataset YAML file
    """
    # Download dataset using kagglehub
    print("Downloading chest CT scan dataset...")
    !pip install -q kagglehub
    import kagglehub

    # Download latest version of the dataset
    data_path = kagglehub.dataset_download("mohamedhanyyy/chest-ctscan-images")
    print(f"Dataset downloaded to: {data_path}")

    # Set output path
    output_path = os.path.join(paths['data_dir'], 'processed_dataset')

    # Prepare the dataset
    yaml_path = prepare_chest_ct_dataset(data_path, output_path)

    return yaml_path

In [ ]:
def train_model(data_yaml, epochs=50):
    """
    Train the YOLO12n model on the prepared dataset

    Args:
        data_yaml: Path to the dataset YAML file
        epochs: Number of training epochs

    Returns:
        YOLO: Trained model
    """
    # Train the model
    model = train_yolo_model(
        data_yaml=data_yaml,
        pretrained_weights='yolo12n.pt',
        epochs=epochs,
        patience=10,
        img_size=640,
        batch_size=16
    )

    # Validate the model
    validate_yolo_model(model, data_yaml)

    return model

In [ ]:
def evaluate_model(model, data_yaml):
    """
    Evaluate the model and visualize results

    Args:
        model: Trained YOLO model
        data_yaml: Path to the dataset YAML file
    """
    # Load dataset configuration
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)

    # Get path to test images
    test_folder = os.path.join(data_config['path'], data_config['val'])

    # Visualize predictions on test images
    visualize_predictions(model, test_folder, num_samples=6)

    # Create confusion matrix
    create_confusion_matrix(model, data_yaml)

In [ ]:
def run_complete_workflow(epochs=50):
    """
    Run the complete workflow from dataset preparation to model evaluation

    Args:
        epochs: Number of training epochs

    Returns:
        YOLO: Trained model
    """
    # 1. Process dataset
    print("Step 1: Processing dataset...")
    data_yaml = process_dataset()

    # 2. Train model
    print("\nStep 2: Training model...")
    model = train_model(data_yaml, epochs=epochs)

    # 3. Evaluate model
    print("\nStep 3: Evaluating model...")
    evaluate_model(model, data_yaml)

    # 4. Test on sample images
    print("\nStep 4: Testing on sample images...")
    test_model_on_samples(model, data_yaml, num_samples=10)

    # 5. Demo interface
    print("\nStep 5: Creating demo interface...")
    create_demo_interface(model)

    return model


In [ ]:
# Run the complete workflow
model = run_complete_workflow(epochs=50)


In [ ]:
# Process the dataset
data_yaml = process_dataset()

In [ ]:
# Train the model
model = train_model(data_yaml, epochs=50)

In [ ]:
# Evaluate the model
evaluate_model(model, data_yaml)

In [ ]:
# Test the model on sample images (5-10 images of cancer and non-cancer)
test_model_on_samples(model, data_yaml, num_samples=10)

In [ ]:
# Create demo interface for uploading and testing new images
create_demo_interface(model)

In [ ]:
def test_with_specific_images(model, image_paths, conf_threshold=0.25):
    """
    Test the model on specific images

    Args:
        model: Trained YOLO model
        image_paths: List of paths to test images
        conf_threshold: Confidence threshold for predictions
    """
    for image_path in image_paths:
        print(f"\nTesting on: {os.path.basename(image_path)}")
        results = predict_single_image(model, image_path, conf_threshold)

        # Print detection results
        boxes = results.boxes
        if len(boxes) == 0:
            print("No detections found.")
        else:
            for i, box in enumerate(boxes):
                conf = float(box.conf[0])
                cls_id = int(box.cls[0])
                cls_name = results.names[cls_id]
                cancer_status = "CANCER" if cls_name != "normal" else "NON-CANCER"
                print(f"Detection {i+1}: {cls_name} ({cancer_status}) with confidence {conf:.2f}")

In [ ]:
def save_model(model, filepath):
    """
    Save the trained model

    Args:
        model: Trained YOLO model
        filepath: Path to save the model
    """
    print(f"Saving model to {filepath}...")
    model.export(format="onnx")  # Save in ONNX format for wider compatibility

    # Also save in PyTorch format
    torch.save(model.model.state_dict(), filepath)
    print(f"Model saved to {filepath}")

def load_model(weights_path):
    """
    Load a trained model

    Args:
        weights_path: Path to the saved model weights

    Returns:
        YOLO: Loaded model
    """
    print(f"Loading model from {weights_path}...")
    model = YOLO(weights_path)
    print("Model loaded successfully!")
    return model

In [ ]:
def calculate_performance_metrics(model, data_yaml):
    """
    Calculate and visualize detailed performance metrics

    Args:
        model: Trained YOLO model
        data_yaml: Path to the dataset YAML file

    Returns:
        dict: Detailed performance metrics
    """
    print("Calculating performance metrics...")

    # Run validation
    results = model.val(data=data_yaml)

    # Extract metrics
    metrics = {
        'precision': results.box.maps[0],  # mAP@0.5
        'recall': results.box.mar[0],      # mAR@0.5
        'mAP50': results.box.map50,        # mAP@0.5
        'mAP50-95': results.box.map,       # mAP@0.5:0.95
        'f1': 2 * (results.box.maps[0] * results.box.mar[0]) /
              (results.box.maps[0] + results.box.mar[0] + 1e-16)  # F1 score
    }

    # Load class names
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)
    class_names = data_config['names']

    # Get metrics per class
    class_metrics = {}
    for i, name in enumerate(class_names):
        try:
            precision = results.box.maps_per_class[i] if i < len(results.box.maps_per_class) else 0
            recall = results.box.mar_per_class[i] if i < len(results.box.mar_per_class) else 0
            class_metrics[name] = {
                'precision': precision,
                'recall': recall,
                'f1': 2 * (precision * recall) / (precision + recall + 1e-16)
            }
        except (IndexError, ZeroDivisionError):
            class_metrics[name] = {'precision': 0, 'recall': 0, 'f1': 0}

    # Print metrics
    print("\nOverall Metrics:")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"mAP@0.5: {metrics['mAP50']:.4f}")
    print(f"mAP@0.5:0.95: {metrics['mAP50-95']:.4f}")

    print("\nMetrics per Class:")
    for cls, cls_metrics in class_metrics.items():
        print(f"{cls}:")
        print(f"  Precision: {cls_metrics['precision']:.4f}")
        print(f"  Recall: {cls_metrics['recall']:.4f}")
        print(f"  F1 Score: {cls_metrics['f1']:.4f}")

    # Plot metrics
    plt.figure(figsize=(10, 6))

    # Plot per-class metrics
    class_names_for_plot = list(class_metrics.keys())
    precision_values = [class_metrics[cls]['precision'] for cls in class_names_for_plot]
    recall_values = [class_metrics[cls]['recall'] for cls in class_names_for_plot]
    f1_values = [class_metrics[cls]['f1'] for cls in class_names_for_plot]

    x = np.arange(len(class_names_for_plot))
    width = 0.25

    plt.bar(x - width, precision_values, width, label='Precision')
    plt.bar(x, recall_values, width, label='Recall')
    plt.bar(x + width, f1_values, width, label='F1 Score')

    plt.xlabel('Class')
    plt.ylabel('Score')
    plt.title('Performance Metrics by Class')
    plt.xticks(x, class_names_for_plot, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return {'overall': metrics, 'per_class': class_metrics}


In [ ]:
# Calculate and display detailed performance metrics
metrics = calculate_performance_metrics(model, data_yaml)